In [1]:
using StaticArrays, DifferentialEquations, DynamicalSystems, CairoMakie, JLD

In [10]:
function sigma(x)
    return 1.0 / ( 1.0 + exp( -10.0 * ( x  - ( - 0.25 ) ) ) )
end

function HR(u, p, t)
        
    a, b, c, d, s, xr, r,  I, vs, chem_link, el_link  = p
    x1, y1, z1, x2, y2, z2 = u
    
    du1 = y1 + b * x1 ^ 2 - a * x1 ^3 - z1 + I - chem_link * ( x1 - vs ) * sigma(x2) + el_link * ( x2 - x1 )
    du2 = c - d * x1 ^2 - y1
    du3 = r * ( s * ( x1 - xr ) - z1 )
    
    du4 = y2 + b * x2 ^ 2 - a * x2 ^3 - z2 + I - chem_link * ( x2 - vs ) * sigma(x1) + el_link * ( x1 - x2 )
    du5 = c - d * x2 ^2 - y2
    du6 = r * ( s * ( x2 - xr ) - z2 )
    return SVector(du1, du2, du3,
                    du4, du5, du6)
end

HR (generic function with 1 method)

In [11]:
function output(idx, parameter, u0)
    println("index: $idx; chemical_link: $parameter");flush(stdout)
    println("u0: $u0");flush(stdout)
end
function output(Λ, u0)
    println("Λ: $Λ");flush(stdout)
    println("last point: $u0");flush(stdout)
end
function separate()
    println("----------------");flush(stdout)
    println("  ");flush(stdout)
end

separate (generic function with 1 method)

In [12]:
a = 1.; b = 3.; c = 1.; d = 5.;
xr = -1.6; r = 0.01; s = 5.; I = 4.; xv = 2.;
electrical_link = 0.01

0.01

In [13]:
tf = 20000.0
tt = 20000.0
tstep = 0.01
integ_setting = (alg = Vern9(), adaptive = false, dt = 0.01);

In [14]:
# chemical link
start_p = -0.02
finish_p = 0.025
p_range = range(start_p, finish_p, length = 500);

In [18]:
Λs = zeros(length(p_range), 6)
u0s = Λs;

In [19]:
for (idx, chemical) in enumerate(p_range)

    if idx == 1
        global u0 = SA[-2.5, 0.0, 0.0, -1.5, 0.0, 0.0]
    end

    output(idx, chemical, u0)

    p = SA[a, b, c, d, s, xr, r, I, xv, chemical, electrical_link]
    ds = ContinuousDynamicalSystem(HR,  u0, p)

    tr = trajectory(ds, 500.0, Δt = tstep; Ttr = tt, diffeq = integ_setting)
    u0 = tr[end]; ds = ContinuousDynamicalSystem(HR,  u0, p)
    
    Λ = lyapunovspectrum(ds, tf; diffeq = integ_setting)
    output(Λ, u0)
    Λs[idx, :] = Λ
    u0s[idx, :] = u0
    separate()
    
end

index: 1; chemical_link: -0.02
u0: [-2.5, 0.0, 0.0, -1.5, 0.0, 0.0]
Λ: [0.004673415615646107, 4.193763101214809e-7, -0.0057957709044824165, -0.012476906057788305, -7.730609310678204, -9.768377674041005]
last point: [-0.9100580567375729, -3.2078501443040177, 4.02153916551849, -0.9055943802477914, -3.1914110446650947, 3.9995492074827372]
----------------
  
index: 2; chemical_link: -0.019909819639278556
u0: [-0.9100580567375729, -3.2078501443040177, 4.02153916551849, -0.9055943802477914, -3.1914110446650947, 3.9995492074827372]
Λ: [0.004494830080646378, 0.00015009378691022065, -0.00538057567186236, -0.012961822425799326, -7.709031476090463, -9.791847896689811]
last point: [0.2778149238322789, 0.6558544671593552, 4.0904990932991465, -0.38778913453385633, -0.2683540813543722, 4.047924593917831]
----------------
  
index: 3; chemical_link: -0.019819639278557115
u0: [0.2778149238322789, 0.6558544671593552, 4.0904990932991465, -0.38778913453385633, -0.2683540813543722, 4.047924593917831]
Λ: [